<a href="https://colab.research.google.com/github/prasadshreyas/CS7290-Causal-ML-PyTorch/blob/main/TMNIST_MNIST_demo_markdown.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing the libraries
```python

import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import os
from torchvision.transforms import ToTensor, Lambda

from pyro.contrib.examples.util import MNIST
import torch.nn as nn
import torchvision.transforms as transforms

import pyro
import pyro.distributions as dist
import pyro.contrib.examples.util  
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
```


```python
assert pyro.__version__.startswith('1.8.1') # make sure we have Pyro 1.8.1
pyro.distributions.enable_validation(False) 
pyro.set_rng_seed(0) # make results reproducible
# Enable smoke test - run the notebook cells on CI.
smoke_test = 'CI' in os.environ 

```

A custom dataloader for T-MNIST Dataset. [link](https://raw.githubusercontent.com/prasadshreyas/CS7290-Causal-ML-PyTorch/main/data/TMNIST/TMNIST_Data.csv)

```python

class TMNISTDataset(Dataset):
    '''
    Typeface MNIST Dataset

    Input: 'path/to/csv/file' (Required)
    Output: torch.utils.data.Dataset
    
    '''
    def __init__(self, csv_file, root_dir='', transform=None, target_transform = Lambda(lambda y: torch.zeros(
    10, dtype=torch.float).scatter_(dim=0, index=torch.tensor(y), value=1))):
        '''
        Args: csv_file (string): Path to the csv file with annotations.    
        '''
        self.tmnist_frame = pd.read_csv(csv_file) # read the csv file
        self.transform = transform # transform the data
        self.root_dir = root_dir 
        self.target_transform = target_transform # transform the target into one-hot vector of size 10
    
    def __len__(self):
        return len(self.tmnist_frame) # return the length of the dataset
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist() # if the index is tensor, convert it to list
        
        digits = self.tmnist_frame.iloc[idx, 2:] # get the digits from the csv file
        digits = np.array([digits])/255 # normalize the digits
        digits = digits.reshape(28,28) # reshape the digits to 28x28 as the original MNIST dataset does
        digits = digits.astype('float32') 
        
        label = self.tmnist_frame.iloc[idx , 1]
        label = np.array([label]) # get the label from the csv file
        label = label.astype('int')
        
        # is_hand is a boolean variable which is True if the digit is hand written and False if it is not.
        # Since, we are using TMNIST, we will set is_hand to False (0)
        is_hand = np.array([0]) 
        is_hand = hand.astype('int')

        if self.transform:
            digits = self.transform(digits)
        return digits, int(label.squeeze()), int(is_hand.squeeze()) # return the digits and the label

```